In [ ]:
import warnings
warnings.filterwarnings('ignore')

import h3
import json
import time

import numpy as np
import pandas as pd
import seaborn as sns
from loguru import logger
import matplotlib.pyplot as plt
from snowflake.snowpark import Session

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import KNNImputer
import xgboost as xgb
import lightgbm as lgb


from utils.geospatial_tools import GeoSpatialTools
from utils.snowflake_mlops import SnowflakeMLOpsManager

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.style.use('seaborn-v0_8-darkgrid')


__geo_tools = GeoSpatialTools()

# 1. Feature Processing & EDA

## 0. Load All Data Sources

In [ ]:
# Load all data sources
raw_train = pd.read_csv('files/train.csv')
raw_test = pd.read_csv('files/test.csv')
raw_base = pd.read_csv('files/h8_summary_hours.csv')
df_points = pd.read_csv('files/points_of_interest_aggregated.csv')

with open('files/point_interest_coordinates.json', 'r') as f:
    json_points_coord = json.load(f)

# Process base data
base = raw_base.drop('Unnamed: 0', axis=1)
base = base.groupby('hex_id').agg({
    'total_hours': 'sum', 'unique_devices': 'sum', 'unique_days': 'sum',
    'work_h1': 'sum', 'work_h2': 'sum', 'work_h3': 'sum', 'work_h4': 'sum',
    'wn_h1': 'sum', 'wn_h2': 'sum', 'wn_h3': 'sum', 'wn_h4': 'sum'
}).reset_index()

print(f"✓ Loaded training data: {raw_train.shape}")
print(f"✓ Loaded test data: {raw_test.shape}")
print(f"✓ Loaded base (activity) data: {base.shape}")
print(f"✓ Loaded POI data: {df_points.shape}")

## 2. Advanced Feature Engineering

In [ ]:
def create_temporal_features(df, base_data):
    """
    Create temporal activity pattern features from device data.
    
    Includes: work/weekend ratios, day/night patterns, activity intensity
    """
    df = df.copy()
    
    # Merge with base activity data
    valid_cols = ['hex_id', 'total_hours', 'work_h1', 'work_h2', 'work_h3', 'work_h4',
                  'wn_h1', 'wn_h2', 'wn_h3', 'wn_h4']
    df = df.merge(base_data[valid_cols], how='left', on='hex_id')
    
    # Mark missing values (imputation needed)
    df['has_activity'] = (~df['total_hours'].isna()).astype(int)
    df['missing'] = df['total_hours'].isna().astype(int)
    
    # Early morning, morning, afternoon, evening, night activity
    df['early_morning'] = df['work_h1'] + df['wn_h1']  # 0-5 hours
    df['morning'] = df['work_h2'] + df['wn_h2']         # 6-11 hours
    df['afternoon'] = df['work_h3'] + df['wn_h3']       # 12-17 hours
    df['evening'] = df['work_h4'] + df['wn_h4']         # 18-23 hours
    
    # Weekday vs weekend patterns
    df['weekday_activity'] = df['work_h1'] + df['work_h2'] + df['work_h3'] + df['work_h4']
    df['weekend_activity'] = df['wn_h1'] + df['wn_h2'] + df['wn_h3'] + df['wn_h4']
    
    # Day vs night (night = 18-5, day = 6-17)
    df['daytime_activity'] = df['morning'] + df['afternoon']
    df['nighttime_activity'] = df['early_morning'] + df['evening']
    
    return df


def create_distance_features(df, amenities_coord, geo_tools):
    """
    Create distance-to-POI features.
    """
    df = df.copy()
    df['hex_center'] = df['hex_id'].apply(lambda x: geo_tools.get_h3_center(x))
    
    # Compute distance features
    for poi_type, coor_list in amenities_coord.items():
        col_name = f"dis_to_{poi_type}"
        if coor_list:
            start_time = pd.Timestamp.now()
            df[col_name] = df['hex_center'].apply(
                lambda x, cl=coor_list: geo_tools.hex_distance_from_coordinates_vectorized(hex_center=x, coor_list=cl)
            )
            end_time = pd.Timestamp.now()
        else:
            df[col_name] = np.nan
    
    return df


def create_poi_features(df, poi_data):
    """
    Create POI density and composition features.
    """
    df = df.copy()
    
    # Merge POI data
    df = df.merge(poi_data, how='left', on='hex_id')

    # Fill missing POI counts with 0
    poi_columns = poi_data.columns.difference(['hex_id'])
    df['missing_external'] = df['bank'].apply(lambda x: 1 if pd.isna(x) else 0)
    df[poi_columns] = df[poi_columns].fillna(0)
    
    # Total POI count
    df['total_poi'] = df[poi_columns].sum(axis=1)
    
    # POI density (count / activity as proxy for area importance)
    df['poi_density'] = df['total_poi'] / (df['total_hours'].fillna(1) + 1)
    
    # # Positive vs negative POI ratio
    # df['positive_poi'] = (df[['health', 'education', 'financial', 'leisure']]
    #                       .sum(axis=1))
    # df['pos_neg_poi_ratio'] = (df['positive_poi'] + 1) / (df['negative'] + 1)
    
    # # Essential services
    # df['essential_services'] = df['health'] + df['security'] + df['financial']
    
    return df


def create_geographic_features(df):
    """
    Create geographic/regional features based on H3 hierarchy.
    """
    df = df.copy()
    
    # Regions based on H3 resolution 3
    costa = ['838f0dfffffffff', '838f28fffffffff', '838f2afffffffff', '838f2bfffffffff', 
             '838f2cfffffffff', '838f2efffffffff']
    sierra = ['8366d1fffffffff', '8366d3fffffffff', '838f29fffffffff', '838f2dfffffffff', 
              '838f72fffffffff', '838f76fffffffff']
    oriente = ['8366dafffffffff']
    
    df['h_3'] = df['hex_id'].apply(lambda x: h3.cell_to_parent(x, res=3))
    df['is_costa'] = df['h_3'].isin(costa).astype(int)
    df['is_sierra'] = df['h_3'].isin(sierra).astype(int)
    df['is_oriente'] = df['h_3'].isin(oriente).astype(int)
    df['missing_points'] = df['bank'].apply(lambda x: 1 if pd.isna(x) else 0)

    return df


def engineer_all_features(df_input, base_data, poi_data, json_coords, geo_tools):
    """
    Apply all feature engineering transformations.
    """
    df = df_input.copy()
    
    print("Creating temporal features...")
    df = create_temporal_features(df, base_data)
    
    print("Creating distance features...")
    df = create_distance_features(df, json_coords, geo_tools)
    
    print("Creating POI features...")
    df = create_poi_features(df, poi_data)
    
    print("Creating geographic features...")
    df = create_geographic_features(df)
    
    return df


# Apply feature engineering to training data
X_raw = engineer_all_features(raw_train, base, df_points, json_points_coord, __geo_tools)
print(f"\n✓ Feature engineering complete. Shape: {X_raw.shape}")

## 3. Data Preprocessing & Imputation

In [ ]:
def preprocess_features(df):
    """
    Handle missing values and create derived features.
    """
    df = df.copy()
    
    # Identify rows with missing temporal data
    missing_mask = df['missing'] == 1
    
    # Calculate total hours from hourly components for missing rows
    df.loc[missing_mask, 'total_hours'] = (
        df.loc[missing_mask, 'work_h1'] + df.loc[missing_mask, 'wn_h1'] +
        df.loc[missing_mask, 'work_h2'] + df.loc[missing_mask, 'wn_h2'] +
        df.loc[missing_mask, 'work_h3'] + df.loc[missing_mask, 'wn_h3'] +
        df.loc[missing_mask, 'work_h4'] + df.loc[missing_mask, 'wn_h4']
    )
    
    # Add small constant to avoid division by zero
    df['total_hours'] = df['total_hours'].fillna(0) + 1e-5
    
    # Activity intensity ratios (normalized by total hours)
    df['early_morning_ratio'] = df['early_morning'] / df['total_hours']
    df['morning_ratio'] = df['morning'] / df['total_hours']
    df['afternoon_ratio'] = df['afternoon'] / df['total_hours']
    df['evening_ratio'] = df['evening'] / df['total_hours']
    
    df['weekday_ratio'] = df['weekday_activity'] / df['total_hours']
    df['weekend_ratio'] = df['weekend_activity'] / df['total_hours']
    
    df['daytime_ratio'] = df['daytime_activity'] / df['total_hours']
    df['nighttime_ratio'] = df['nighttime_activity'] / df['total_hours']
    
    # Peak activity hours
    activity_cols = ['early_morning', 'morning', 'afternoon', 'evening']
    df['peak_activity'] = df[activity_cols].max(axis=1)
    df['activity_variance'] = df[activity_cols].var(axis=1).fillna(0)
    
    # Distance statistics
    distance_columns = [col for col in X_raw.columns if col.startswith('dis_to_')]

    df['avg_distance_to_poi'] = df[distance_columns].mean(axis=1)
    df['min_distance_to_poi'] = df[distance_columns].min(axis=1)
    
    return df


# Apply preprocessing
X = preprocess_features(X_raw)

# Handle any remaining NaN values
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].median())

print(f"✓ Preprocessing complete. Final features: {len(numeric_cols)}")
print(f"  Missing values: {X[numeric_cols].isnull().sum().sum()}")

## 4. Feature Selection & Analysis

In [ ]:
# Select only numeric features for modeling
feature_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Remove hex_id from features
if 'hex_id' in feature_cols:
    feature_cols.remove('hex_id')

# Get target
y = X['cost_of_living'].copy()
X_features = X[feature_cols].copy()

# Remove features with zero variance
zero_var_features = X_features.columns[X_features.var() < 1e-6].tolist()
X_features = X_features.drop(columns=zero_var_features)
print(f"Removed {len(zero_var_features)} zero-variance features")

# Correlation analysis
plt.figure(figsize=(14, 8))
correlation = X_features.corr()['cost_of_living' if 'cost_of_living' in X_features.columns else y.name].drop('cost_of_living', errors='ignore').sort_values(ascending=False)

# Show top 15 features by correlation
print("\n📊 Top 15 Features by Correlation with Target:")
print(correlation.head(15))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

correlation.head(10).plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Top 10 Positively Correlated Features')
axes[0].set_xlabel('Correlation')

correlation.tail(5).plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Top 5 Negatively Correlated Features')
axes[1].set_xlabel('Correlation')

plt.tight_layout()
plt.show()

# Select features with correlation > 0.05 (by absolute value)
selected_features = correlation[correlation.abs() > 0].index.tolist()
print(f"\n✓ Selected {len(selected_features)} features with |correlation| > 0.05")

# If too few features, use all numeric features
if len(selected_features) < 5:
    selected_features = feature_cols[:20]  # Use top 20 features
    print(f"  (Using {len(selected_features)} features as fallback)")

X_selected = X_features[selected_features].copy()
print(f"\nFinal feature set shape: {X_selected.shape}")

## 5. Model Training & Cross-Validation

### 5.1 Model Diagnostics & Improvements Applied

In [ ]:
def evaluate_model_with_cv(model, X, y, model_name, cv_folds=5):
    """
    Evaluate model with cross-validation.
    """
    kfold = KFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)
    
    # Cross-validation scores (FIX: only negate once for neg_mean_squared_error)
    start_time = time.time()
    mse_scores = -cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(mse_scores)  # Only negate once, not twice!
    r2_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')
    mae_scores = -cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
    elapsed = time.time() - start_time
    
    results = {
        'Model': model_name,
        'RMSE': rmse_scores.mean(),
        'RMSE_std': rmse_scores.std(),
        'R2': r2_scores.mean(),
        'R2_std': r2_scores.std(),
        'MAE': mae_scores.mean(),
        'MAE_std': mae_scores.std(),
        'Time(s)': elapsed
    }
    return results, rmse_scores

# Initialize models with GPU acceleration where available
print("\n🔧 Initializing models...")
USE_GPU = False
if USE_GPU:
    print("  Using GPU acceleration (CUDA) for XGBoost and LightGBM")
else:
    print("  Using CPU for all models")

models = {
    'Random_Forest': RandomForestRegressor(
        n_estimators=150,
        max_depth=20, 
        min_samples_split=5, 
        min_samples_leaf=2,
        random_state=RANDOM_STATE, 
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.08,
        max_depth=7, 
        min_samples_split=5,
        subsample=0.9,
        random_state=RANDOM_STATE
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=300,
        learning_rate=0.08,
        max_depth=7, 
        subsample=0.9,
        colsample_bytree=0.9,
        min_child_weight=1,
        tree_method='hist',
        device='cpu',
        random_state=RANDOM_STATE
    ),
    'LightGBM': lgb.LGBMRegressor(
        n_estimators=300,
        learning_rate=0.08,
        num_leaves=40,
        max_depth=8,
        min_child_samples=5,
        subsample=0.9,
        subsample_freq=1,
        device='gpu' if USE_GPU else 'cpu',
        random_state=RANDOM_STATE,
        verbose=-1
    ),
}

print("=" * 80)
print("🔄 CROSS-VALIDATION RESULTS (5-Fold)")
print("=" * 80)

cv_results = []
model_scores = {}

for name, model in models.items():
    print(f"\n📍 Training {name}...")
    results, rmse_scores = evaluate_model_with_cv(model, X_selected, y, name, cv_folds=5)
    cv_results.append(results)
    model_scores[name] = rmse_scores
    
    print(f"   RMSE: {results['RMSE']:.6f} (+/- {results['RMSE_std']:.6f})")
    print(f"   R²:   {results['R2']:.6f} (+/- {results['R2_std']:.6f})")
    print(f"   MAE:  {results['MAE']:.6f} (+/- {results['MAE_std']:.6f})")

# Display results
cv_df = pd.DataFrame(cv_results).sort_values('RMSE')
print("\n" + "=" * 80)
print("📊 MODEL COMPARISON")
print("=" * 80)
print(cv_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

cv_df.plot(x='Model', y='RMSE', kind='bar', ax=axes[0], legend=False, color='steelblue')
axes[0].set_title('RMSE by Model')
axes[0].set_ylabel('RMSE')
axes[0].grid(axis='y', alpha=0.3)

cv_df.plot(x='Model', y='R2', kind='bar', ax=axes[1], legend=False, color='green')
axes[1].set_title('R² by Model')
axes[1].set_ylabel('R² Score')

axes[1].grid(axis='y', alpha=0.3)
# Select best modelprint(f"   CV RMSE: {cv_df.iloc[0]['RMSE']:.6f}")


best_model_name = cv_df.iloc[0]['Model']
print(f"\n✅ Best model: {best_model_name}")

cv_df.plot(x='Model', y='MAE', kind='bar', ax=axes[2], legend=False, color='orange')
print(f"\n✅ Best model: {best_model_name}")
best_model_name = cv_df.iloc[0]['Model']

axes[2].set_title('MAE by Model')
print(f"   CV RMSE: {cv_df.iloc[0]['RMSE']:.6f}")# Select best model

axes[2].set_ylabel('MAE')

axes[2].grid(axis='y', alpha=0.3)
plt.show()

plt.tight_layout()

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

cv_df.plot(x='Model', y='RMSE', kind='bar', ax=axes[0, 0], legend=False, color='steelblue')
axes[0, 0].set_title('RMSE by Model (Lower is Better)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('RMSE')
axes[0, 0].grid(axis='y', alpha=0.3)

cv_df.plot(x='Model', y='R2', kind='bar', ax=axes[0, 1], legend=False, color='green')
axes[0, 1].set_title('R² by Model (Higher is Better)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('R² Score')
axes[0, 1].grid(axis='y', alpha=0.3)

cv_df.plot(x='Model', y='MAE', kind='bar', ax=axes[1, 0], legend=False, color='orange')
axes[1, 0].set_title('MAE by Model (Lower is Better)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('MAE')
axes[1, 0].grid(axis='y', alpha=0.3)

cv_df.plot(x='Model', y='Time(s)', kind='bar', ax=axes[1, 1], legend=False, color='coral')
axes[1, 1].set_title('Training Time (5-Fold CV)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Select best model
best_model_name = cv_df.iloc[0]['Model']
print(f"\n✅ Best model: {best_model_name}")
print(f"   CV RMSE: {cv_df.iloc[0]['RMSE']:.6f}")
print(f"   CV Time: {cv_df.iloc[0]['Time(s)']:.2f}s")

# Show speedup if GPU was used
if USE_GPU:
    print(f"\n🚀 GPU acceleration enabled - training should be significantly faster!")

## 6. Hyperparameter Tuning for Best Model

In [ ]:
#
import os

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE", "AI_CORTEX_DEVELOPER_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE", "ai_project_wh"),
    "database": os.getenv("SNOWFLAKE_DATABASE", "ai_project"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA", "mlops"),
}

snowflake_session = Session.builder.configs(connection_parameters).create()

__snow_mlops = SnowflakeMLOpsManager(snowflake_session, experiment_name=best_model_name)


In [ ]:
# Select model for tuning (use the best one from CV)
best_model = models[best_model_name]

# Split data for evaluation (needed for log_run)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=RANDOM_STATE
)

# Define metrics function for experiment tracking
def metrics_fn(y_true, y_pred):
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2_score": float(r2_score(y_true, y_pred)),
    }

# Define hyperparameter search space
if best_model_name == 'XGBoost':
    param_dist = {
        'n_estimators': [100, 150, 200, 250],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [4, 6, 8, 10],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'gamma': [0, 0.1, 0.5],
    }
elif best_model_name == 'LightGBM':
    param_dist = {
        'n_estimators': [100, 150, 200, 250],
        'learning_rate': [0.05, 0.1, 0.15],
        'num_leaves': [20, 31, 50],
        'max_depth': [4, 6, 8],
        'min_child_samples': [5, 10, 20],
    }
elif best_model_name == 'Random_Forest':
    param_dist = {
        'n_estimators': [150, 200, 300, 500],
        'max_depth': [15, 20, 25, 30],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
    }
else:  # Gradient Boosting
    param_dist = {
        'n_estimators': [100, 150, 200],
        'learning_rate': [0.05, 0.1, 0.15],
        'max_depth': [4, 6, 8],
        'min_samples_split': [2, 5],
    }

print(f"🔍 Tuning {best_model_name} with Experiment Tracking...")
print(f"   Parameter search space: {len(param_dist)} parameters")

# Generate and sample parameter combinations
import itertools
param_combinations = list(itertools.product(*param_dist.values()))
param_names = list(param_dist.keys())
n_iter = min(20, len(param_combinations))
sampled_indices = np.random.RandomState(RANDOM_STATE).choice(
    len(param_combinations), size=n_iter, replace=False
)

best_metrics = {"rmse": float('inf')}
best_params = None
best_run_name = None

print("=" * 80)
print(f"🚀 Running {n_iter} hyperparameter experiments with Snowflake MLOps\n")

# Loop through parameter combinations and log runs
for i, idx in enumerate(sampled_indices):
    params = dict(zip(param_names, param_combinations[idx]))
    run_name = f"run_{i:02d}_{best_model_name}"
    
    print(f"📌 Run {i+1}/{n_iter}: {run_name} | Params: {params}")
    
    # Instantiate fresh model
    if best_model_name == 'XGBoost':
        model = xgb.XGBRegressor(**params, tree_method='hist', device='cpu', random_state=RANDOM_STATE)
    elif best_model_name == 'LightGBM':
        model = lgb.LGBMRegressor(**params, device='cpu', random_state=RANDOM_STATE, verbose=-1)
    elif best_model_name == 'Random Forest':
        model = RandomForestRegressor(**params, random_state=RANDOM_STATE, n_jobs=-1)
    else:  # Gradient Boosting
        model = GradientBoostingRegressor(**params, random_state=RANDOM_STATE)
    
    try:
        # Log run to Snowflake MLOps
        metrics = __snow_mlops.log_run(
            run_name=run_name,
            model=model,
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            params=params,
            metrics_fn=metrics_fn,
            log_model=False
        )
        
        print(f"   ✅ RMSE: {metrics['rmse']:.6f} | MAE: {metrics['mae']:.6f} | R²: {metrics['r2_score']:.6f}\n")
        
        # Track best run
        if metrics['rmse'] < best_metrics['rmse']:
            best_metrics = metrics
            best_params = params
            best_run_name = run_name
            
    except Exception as e:
        print(f"   ❌ Error: {str(e)}\n")

print("=" * 80)
print(f"\n✅ Best hyperparameters found:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

best_cv_rmse = best_metrics['rmse']
print(f"\n   Best RMSE: {best_cv_rmse:.6f}")

# Use the best model from tuning
if best_model_name == 'XGBoost':
    best_model_tuned = xgb.XGBRegressor(**best_params, tree_method='hist', device='cpu', random_state=RANDOM_STATE)
elif best_model_name == 'LightGBM':
    best_model_tuned = lgb.LGBMRegressor(**best_params, device='cpu', random_state=RANDOM_STATE, verbose=-1)
elif best_model_name == 'Random Forest':
    best_model_tuned = RandomForestRegressor(**best_params, random_state=RANDOM_STATE, n_jobs=-1)
else:  # Gradient Boosting
    best_model_tuned = GradientBoostingRegressor(**best_params, random_state=RANDOM_STATE)

best_model_tuned.fit(X_selected, y)

## 7. Feature Importance & Model Insights

In [ ]:
# Train best model on full dataset for feature importance
# best_model_tuned.fit(X_selected, y)

# Get feature importance
if hasattr(best_model_tuned, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_selected.columns,
        'importance': best_model_tuned.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n📊 Top 15 Most Important Features:")
    print(feature_importance.head(15).to_string(index=False))
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    top_features = feature_importance.head(15)
    top_features.plot(x='feature', y='importance', kind='barh', ax=axes[0], color='steelblue', legend=False)
    axes[0].set_title('Top 15 Feature Importances')
    axes[0].set_xlabel('Importance')
    
    # Cumulative importance
    cumsum = feature_importance['importance'].cumsum() / feature_importance['importance'].sum()
    axes[1].plot(cumsum.values, marker='o', linestyle='-', linewidth=2)
    axes[1].axhline(y=0.95, color='r', linestyle='--', label='95% explained variance')
    axes[1].set_xlabel('Number of Features')
    axes[1].set_ylabel('Cumulative Importance')
    axes[1].set_title('Cumulative Feature Importance')
    axes[1].grid(alpha=0.3)
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

# Final evaluation on test split
X_train_eval, X_test_eval, y_train_eval, y_test_eval = train_test_split(
    X_selected, y, test_size=0.2, random_state=RANDOM_STATE
)

best_model_tuned.fit(X_train_eval, y_train_eval)
y_pred_eval = best_model_tuned.predict(X_test_eval)

rmse_test = np.sqrt(mean_squared_error(y_test_eval, y_pred_eval))
r2_test = r2_score(y_test_eval, y_pred_eval)
mae_test = mean_absolute_error(y_test_eval, y_pred_eval)

print("\n" + "=" * 80)
print("📈 FINAL MODEL EVALUATION (80/20 Test Split)")
print("=" * 80)
print(f"RMSE: {rmse_test:.6f}")
print(f"R² Score: {r2_test:.6f}")
print(f"MAE: {mae_test:.6f}")

# Residual analysis
residuals = y_test_eval - y_pred_eval

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Residuals vs predictions
axes[0, 0].scatter(y_pred_eval, residuals, alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residual Plot')
axes[0, 0].grid(alpha=0.3)

# Histogram of residuals
axes[0, 1].hist(residuals, bins=20, edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Residuals Distribution')
axes[0, 1].grid(alpha=0.3)

# Actual vs Predicted
axes[1, 0].scatter(y_test_eval, y_pred_eval, alpha=0.6)
axes[1, 0].plot([y_test_eval.min(), y_test_eval.max()], 
                [y_test_eval.min(), y_test_eval.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Values')
axes[1, 0].set_ylabel('Predicted Values')
axes[1, 0].set_title(f'Actual vs Predicted (R² = {r2_test:.4f})')
axes[1, 0].grid(alpha=0.3)

# Q-Q plot for residuals
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
from snowflake.ml.model.model_signature import infer_signature

# Log the final model to Snowflake MLOps

# Infer the model signature
sig = infer_signature(X_train_eval, y_train_eval)

# Log the model using SnowflakeMLOpsManager
__snow_mlops.exp.log_model(
    model=best_model_tuned,
    model_name=f"{best_model_name}_final",
    signatures={"predict": sig},
)

In [ ]:
__snow_mlops.exp.end_run()

## 8. Generate Predictions for Test Set

In [ ]:
# Prepare test data with same feature engineering pipeline
print("🔄 Preparing test data...")
X_test_raw = engineer_all_features(raw_test, base, df_points, json_points_coord, __geo_tools)
X_test_processed = preprocess_features(X_test_raw)

# Handle NaN values in test set
X_test_processed[numeric_cols] = X_test_processed[numeric_cols].fillna(
    X[numeric_cols].median()
)

# Select same features as training
X_test_selected = X_test_processed[selected_features].copy()

print(f"✓ Test set shape: {X_test_selected.shape}")

# Ensure test set has same features as training
for col in selected_features:
    if col not in X_test_selected.columns:
        X_test_selected[col] = X[col].median()

# Train final model on full training data
print(f"\n🎯 Training final {best_model_name} on full training set...")
best_model_final = best_model_tuned
best_model_final.fit(X_selected, y)

# Make predictions on test set
y_pred_test = best_model_final.predict(X_test_selected)

# Ensure predictions are within valid range (0-1)
y_pred_test = np.clip(y_pred_test, 0, 1)

print(f"\n✅ Generated predictions for {len(y_pred_test)} test samples")
print(f"   Min prediction: {y_pred_test.min():.4f}")
print(f"   Max prediction: {y_pred_test.max():.4f}")
print(f"   Mean prediction: {y_pred_test.mean():.4f}")

# Create submission file
submission = raw_test[['hex_id']].copy()
submission['cost_of_living'] = y_pred_test

print(f"\n📊 Submission preview:")
print(submission.head(10))




In [ ]:
current_timestap = pd.Timestamp.now().strftime("%Y_%m_%d_%H_%M_%S")

iteration_name = f"{best_model_name}_{current_timestap}"
# Save submission
submission.to_csv(f'output_{iteration_name}.csv', index=False)
print(f"\n✅ Saved predictions to 'output_{iteration_name}.csv'")

## 9. Summary & Next Steps

In [ ]:
print("\n" + "=" * 80)
print("🎉 MODELING COMPLETE - SUMMARY")
print("=" * 80)

print(f"""
✅ Data Processing:
   - Loaded training samples: {len(raw_train)}
   - Loaded test samples: {len(raw_test)}
   - Final feature count: {len(selected_features)}

✅ Feature Engineering:
   - Temporal patterns (activity ratios, peak hours)
   - Distance-to-POI features
   - POI density and composition
   - Geographic/regional features

✅ Model Training:
   - Models compared: {len(models)}
   - Best model: {best_model_name}
   - Cross-validation RMSE: {best_cv_rmse:.6f}
   - Test RMSE: {rmse_test:.6f}
   - Test R²: {r2_test:.6f}

📈 Performance Improvements:
   - Previous RMSE baseline: 0.1634
   - Current test RMSE: {rmse_test:.6f}
   - Improvement: {((0.1634 - rmse_test) / 0.1634 * 100):.2f}%

📁 Output Files:
   - Predictions: 'output_final.csv'
   - Model: Ready for serialization and deployment

🚀 Next Steps:
   1. Save model artifacts for production deployment
   2. Create API wrapper for real-time predictions
   3. Set up monitoring and retraining pipelines
   4. Consider ensemble methods if further gains needed
   5. Validate on holdout test set
""")

print("=" * 80)

In [ ]:
best = pd.read_csv('output.csv')
sub1 = pd.read_csv(f'output_{iteration_name}.csv')
sub2 = pd.read_csv('output_rf_cv_2.csv')

best_weight = 0.33
sub1_weight = 0.65
sub2_weight = 0.02

best['cost_of_living'] = (best_weight * best['cost_of_living']) + (sub1_weight * sub1['cost_of_living']) + (sub2_weight * sub2['cost_of_living']) 

best.to_csv(f'output_ensemble_{iteration_name}_v3.csv', index=False)